In [25]:
import pandas as pd
from cmdstanpy import CmdStanModel

# Read the CSV data
df = pd.read_csv('input/data-salary-2.csv')
df.head()

stan_data = {
    'N': len(df),
    'C': df['CID'].nunique(),
    'X': df['X'].values,
    'Y': df['Y'].values,
    'n2c': df['CID'].values
}

In [26]:
stan_file = 'model/model8-1.stan'
with open(stan_file, 'r') as f:
    print(f.read())

data {
  int N;
  vector[N] X;
  vector[N] Y;
}

parameters {
  real a;
  real b;
  real<lower=0> s_y;
}

model {
  Y[1:N] ~ normal(a + b*X[1:N], s_y);
}



In [32]:
# Prepare the Stan model
stan_model = CmdStanModel(stan_file=stan_file)
fit = stan_model.sample(data=stan_data)

14:30:06 - cmdstanpy - INFO - CmdStan start processing
chain 1 |          | 00:00 Status


chain 1 |██████████| 00:00 Sampling completed
chain 2 |██████████| 00:00 Sampling completed
chain 3 |██████████| 00:00 Sampling completed
chain 4 |██████████| 00:00 Sampling completed


14:30:06 - cmdstanpy - INFO - CmdStan done processing.


In [45]:
quantile_pairs = samples_df[['a', 'b', 's_y']].quantile([0.025, 0.975])
print(quantile_pairs)
for param in ['a', 'b', 's_y']:
    print(f"{param}: mean: {samples_df[param].mean():.2f}, 95% CI: [{quantile_pairs.loc[0.025, param]:.2f}, {quantile_pairs.loc[0.975, param]:.2f}]")

               a         b       s_y
0.025  32.624705  0.830763  5.463993
0.975  42.372145  1.385567  8.520984
a: mean: 37.60, 95% CI: [32.62, 42.37]
b: mean: 1.11, 95% CI: [0.83, 1.39]
s_y: mean: 6.84, 95% CI: [5.46, 8.52]


---

In [46]:
stan_file = 'model/model8-2.stan'
with open(stan_file, 'r') as f:
    print(f.read())

data {
  int N;
  int C;
  vector[N] X;
  vector[N] Y;
  array[N] int<lower=1, upper=C> n2c;
}

parameters {
  vector[C] a;
  vector[C] b;
  real<lower=0> s_y;
}

model {
  Y[1:N] ~ normal(a[n2c] + b[n2c] .* X[1:N], s_y);
}



In [50]:
# Prepare the Stan model
stan_model = CmdStanModel(stan_file=stan_file)
fit = stan_model.sample(data=stan_data)
samples_df = fit.draws_pd()

14:42:10 - cmdstanpy - INFO - CmdStan start processing
chain 1 |          | 00:00 Status


chain 1 |████      | 00:00 Iteration:  700 / 2000 [ 35%]  (Warmup)


chain 1 |█████████▌| 00:00 Iteration: 1800 / 2000 [ 90%]  (Sampling)


chain 1 |██████████| 00:00 Sampling completed                       
chain 2 |██████████| 00:00 Sampling completed                       
chain 3 |██████████| 00:00 Sampling completed                       
chain 4 |██████████| 00:00 Sampling completed                       


14:42:10 - cmdstanpy - INFO - CmdStan done processing.


In [52]:
quantile_pairs = samples_df[['a[1]', 'a[2]', 'a[3]', 'a[4]', 'b[1]', 'b[2]', 'b[3]', 'b[4]', 's_y']].quantile([0.025, 0.975])
# print(quantile_pairs)
for param in ['a[1]', 'a[2]', 'a[3]', 'a[4]', 'b[1]', 'b[2]', 'b[3]', 'b[4]', 's_y']:
    mean_val = samples_df[param].mean()
    ci_low = quantile_pairs.loc[0.025, param]
    ci_high = quantile_pairs.loc[0.975, param]
    print(f"{param}: mean: {mean_val:.2f}, 95% CI: [{ci_low:.2f}, {ci_high:.2f}]")

a[1]: mean: 38.71, 95% CI: [36.07, 41.38]
a[2]: mean: 32.87, 95% CI: [29.65, 36.14]
a[3]: mean: 31.48, 95% CI: [24.68, 38.25]
a[4]: mean: 74.79, 95% CI: [41.13, 106.64]
b[1]: mean: 0.75, 95% CI: [0.58, 0.92]
b[2]: mean: 1.99, 95% CI: [1.75, 2.23]
b[3]: mean: 1.24, 95% CI: [0.91, 1.58]
b[4]: mean: -0.09, 95% CI: [-1.38, 1.25]
s_y: mean: 2.73, 95% CI: [2.12, 3.54]


---

In [54]:
# Hierarchical Model
stan_file = 'model/model8-3.stan'
with open(stan_file, 'r') as f:
    print(f.read())

data {
  int N;
  int C;
  vector[N] X;
  vector[N] Y;
  array[N] int<lower=1, upper=C> n2c;
}

parameters {
  real a_field;
  real b_field;
  vector[C] a_diff;
  vector[C] b_diff;
  real<lower=0> s_a;
  real<lower=0> s_b;
  real<lower=0> s_y;
}

transformed parameters {
  vector[C] a = a_field + a_diff[1:C];
  vector[C] b = b_field + b_diff[1:C];
}

model {
  a_diff[1:C] ~ normal(0, s_a);
  b_diff[1:C] ~ normal(0, s_b);
  Y[1:N] ~ normal(a[n2c] + b[n2c] .* X[1:N], s_y);
}



In [55]:
# Prepare the Stan model
stan_model = CmdStanModel(stan_file=stan_file)
fit = stan_model.sample(data=stan_data)
samples_df = fit.draws_pd()

14:44:28 - cmdstanpy - INFO - CmdStan start processing
chain 1 |          | 00:00 Status




chain 1 |▉         | 00:00 Iteration:    1 / 2000 [  0%]  (Warmup)
chain 1 |█▊        | 00:00 Iteration:  200 / 2000 [ 10%]  (Warmup)


chain 1 |███▏      | 00:00 Iteration:  500 / 2000 [ 25%]  (Warmup)


chain 1 |█████▉    | 00:00 Iteration: 1001 / 2000 [ 50%]  (Sampling)


chain 1 |███████▋  | 00:00 Iteration: 1400 / 2000 [ 70%]  (Sampling)


chain 1 |█████████▌| 00:00 Iteration: 1800 / 2000 [ 90%]  (Sampling)


chain 1 |██████████| 00:00 Sampling completed                       
chain 2 |██████████| 00:00 Sampling completed                       
chain 3 |██████████| 00:00 Sampling completed                       
chain 4 |██████████| 00:00 Sampling completed                       


14:44:29 - cmdstanpy - INFO - CmdStan done processing.
14:44:29 - cmdstanpy - WARNING - Some chains may have failed to converge.
	Chain 1 had 1 iterations at max treedepth (0.1%)
	Chain 2 had 1 divergent transitions (0.1%)
	Chain 2 had 1 iterations at max treedepth (0.1%)
	Use the "diagnose()" method on the CmdStanMCMC object to see further information.


In [59]:
quantile_pairs = samples_df[['a_field', 'b_field', 's_a', 's_b', 's_y']].quantile([0.025, 0.975])
for param in ['a_field', 'b_field', 's_a', 's_b', 's_y']:
    mean_val = samples_df[param].mean()
    ci_low = quantile_pairs.loc[0.025, param]
    ci_high = quantile_pairs.loc[0.975, param]
    print(f"{param}: mean: {mean_val:.2f}, 95% CI: [{ci_low:.2f}, {ci_high:.2f}]")

a_field: mean: 38.65, 95% CI: [21.26, 65.47]
b_field: mean: 1.24, 95% CI: [-0.47, 2.90]
s_a: mean: 14.77, 95% CI: [1.22, 60.07]
s_b: mean: 1.19, 95% CI: [0.31, 4.40]
s_y: mean: 2.87, 95% CI: [2.20, 3.73]


In [ ]:
quantile_pairs = samples_df[['a_field', 'b_field', 's_a', 's_b', 's_y']].quantile([0.025, 0.975])
for param in ['a_field', 'b_field', 's_a', 's_b', 's_y']:
    mean_val = samples_df[param].median()
    ci_low = quantile_pairs.loc[0.025, param]
    ci_high = quantile_pairs.loc[0.975, param]
    print(f"{param}: median: {mean_val:.1f}, 95% CI: [{ci_low:.1f}, {ci_high:.1f}]")

a_field: median: 36.7, 95% CI: [13.9, 60.7]
b_field: median: 1.2, 95% CI: [-0.4, 2.8]
s_a: median: 9.6, 95% CI: [1.4, 76.4]
s_b: median: 0.8, 95% CI: [0.3, 4.6]
s_y: median: 2.8, 95% CI: [2.2, 3.8]


---

In [60]:
# Hierarchical Model
stan_file = 'model/model8-4.stan'
with open(stan_file, 'r') as f:
    print(f.read())

data {
  int N;
  int C;
  vector[N] X;
  vector[N] Y;
  array[N] int<lower=1, upper=C> n2c;
}

parameters {
  real a_field;
  real b_field;
  vector[C] a;
  vector[C] b;
  real<lower=0> s_a;
  real<lower=0> s_b;
  real<lower=0> s_y;
}

model {
  a[1:C] ~ normal(a_field, s_a);
  b[1:C] ~ normal(b_field, s_b);
  Y[1:N] ~ normal(a[n2c] + b[n2c] .* X[1:N], s_y);
}



In [61]:
# Prepare the Stan model
stan_model = CmdStanModel(stan_file=stan_file)
fit = stan_model.sample(data=stan_data)
samples_df = fit.draws_pd()

14:48:53 - cmdstanpy - INFO - CmdStan start processing
chain 1 |          | 00:00 Status



chain 1 |██▋       | 00:00 Iteration:  400 / 2000 [ 20%]  (Warmup)


chain 1 |████████▏ | 00:00 Iteration: 1500 / 2000 [ 75%]  (Sampling)

chain 1 |██████████| 00:00 Sampling completed                       
chain 2 |██████████| 00:00 Sampling completed                       
chain 3 |██████████| 00:00 Sampling completed                       
chain 4 |██████████| 00:00 Sampling completed                       


14:48:53 - cmdstanpy - INFO - CmdStan done processing.
14:48:53 - cmdstanpy - WARNING - Some chains may have failed to converge.
	Chain 1 had 6 divergent transitions (0.6%)
	Chain 2 had 2 divergent transitions (0.2%)
	Chain 4 had 23 divergent transitions (2.3%)
	Use the "diagnose()" method on the CmdStanMCMC object to see further information.


In [65]:
quantile_pairs = samples_df[['a_field', 'b_field', 's_a', 's_b', 's_y']].quantile([0.025, 0.975])
for param in ['a_field', 'b_field', 's_a', 's_b', 's_y']:
    mean_val = samples_df[param].median()
    ci_low = quantile_pairs.loc[0.025, param]
    ci_high = quantile_pairs.loc[0.975, param]
    print(f"{param}: median: {mean_val:.1f}, 95% CI: [{ci_low:.1f}, {ci_high:.1f}]")

a_field: median: 36.7, 95% CI: [13.9, 60.7]
b_field: median: 1.2, 95% CI: [-0.4, 2.8]
s_a: median: 9.6, 95% CI: [1.4, 76.4]
s_b: median: 0.8, 95% CI: [0.3, 4.6]
s_y: median: 2.8, 95% CI: [2.2, 3.8]


In [63]:
quantile_pairs = samples_df[['a[1]', 'a[2]', 'a[3]', 'a[4]', 'b[1]', 'b[2]', 'b[3]', 'b[4]', 's_y']].quantile([0.025, 0.975])
# print(quantile_pairs)
for param in ['a[1]', 'a[2]', 'a[3]', 'a[4]', 'b[1]', 'b[2]', 'b[3]', 'b[4]', 's_y']:
    mean_val = samples_df[param].mean()
    ci_low = quantile_pairs.loc[0.025, param]
    ci_high = quantile_pairs.loc[0.975, param]
    print(f"{param}: mean: {mean_val:.2f}, 95% CI: [{ci_low:.2f}, {ci_high:.2f}]")

a[1]: mean: 38.37, 95% CI: [35.15, 41.37]
a[2]: mean: 33.50, 95% CI: [30.12, 36.95]
a[3]: mean: 32.46, 95% CI: [25.45, 38.88]
a[4]: mean: 47.97, 95% CI: [30.10, 80.83]
b[1]: mean: 0.77, 95% CI: [0.59, 0.96]
b[2]: mean: 1.94, 95% CI: [1.68, 2.19]
b[3]: mean: 1.19, 95% CI: [0.88, 1.54]
b[4]: mean: 0.99, 95% CI: [-0.34, 1.71]
s_y: mean: 2.86, 95% CI: [2.21, 3.76]


---

Provide some proper priors to avoid divergent chain

In [ ]:
# Hierarchical Model
stan_file = 'model/model8-4.stan'
with open(stan_file, 'r') as f:
    print(f.read())

data {
  int N;
  int C;
  vector[N] X;
  vector[N] Y;
  array[N] int<lower=1, upper=C> n2c;
}

parameters {
  real a_field;
  real b_field;
  vector[C] a;
  vector[C] b;
  real<lower=0> s_a;
  real<lower=0> s_b;
  real<lower=0> s_y;
}

model {
  a[1:C] ~ normal(a_field, s_a);
  b[1:C] ~ normal(b_field, s_b);
  Y[1:N] ~ normal(a[n2c] + b[n2c] .* X[1:N], s_y);
}



In [71]:
# Prepare the Stan model
stan_model = CmdStanModel(stan_file=stan_file)
fit = stan_model.sample(data=stan_data, adapt_delta=0.999)
samples_df = fit.draws_pd()

14:54:50 - cmdstanpy - INFO - CmdStan start processing
chain 1 |          | 00:00 Status



chain 1 |▉         | 00:00 Iteration:    1 / 2000 [  0%]  (Warmup)


chain 1 |██▋       | 00:00 Iteration:  400 / 2000 [ 20%]  (Warmup)


chain 1 |████▌     | 00:00 Iteration:  800 / 2000 [ 40%]  (Warmup)


chain 1 |██████▎   | 00:00 Iteration: 1100 / 2000 [ 55%]  (Sampling)


chain 1 |███████▋  | 00:00 Iteration: 1400 / 2000 [ 70%]  (Sampling)

chain 1 |█████████ | 00:00 Iteration: 1700 / 2000 [ 85%]  (Sampling)


chain 1 |██████████| 00:02 Sampling completed                       
chain 2 |██████████| 00:02 Sampling completed                       
chain 3 |██████████| 00:01 Sampling completed                       
chain 4 |██████████| 00:01 Sampling completed                       


14:54:52 - cmdstanpy - INFO - CmdStan done processing.
14:54:52 - cmdstanpy - WARNING - Some chains may have failed to converge.
	Chain 1 had 2 divergent transitions (0.2%)
	Chain 2 had 407 iterations at max treedepth (40.7%)
	Use the "diagnose()" method on the CmdStanMCMC object to see further information.


In [70]:
print(fit.diagnose())

Checking sampler transitions treedepth.
Treedepth satisfactory for all transitions.

Checking sampler transitions for divergences.
7 of 1000 (0.70%) transitions ended with a divergence.
These divergent transitions indicate that HMC is not fully able to explore the posterior distribution.
Try increasing adapt delta closer to 1.
If this doesn't remove all divergences, try to reparameterize the model.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory.

Rank-normalized split effective sample size satisfactory for all parameters.

Rank-normalized split R-hat values satisfactory for all parameters.

Processing complete.



In [ ]:
quantile_pairs = samples_df[['a_field', 'b_field', 's_a', 's_b', 's_y']].quantile([0.025, 0.975])
for param in ['a_field', 'b_field', 's_a', 's_b', 's_y']:
    mean_val = samples_df[param].median()
    ci_low = quantile_pairs.loc[0.025, param]
    ci_high = quantile_pairs.loc[0.975, param]
    print(f"{param}: median: {mean_val:.1f}, 95% CI: [{ci_low:.1f}, {ci_high:.1f}]")

a_field: median: 36.7, 95% CI: [13.9, 60.7]
b_field: median: 1.2, 95% CI: [-0.4, 2.8]
s_a: median: 9.6, 95% CI: [1.4, 76.4]
s_b: median: 0.8, 95% CI: [0.3, 4.6]
s_y: median: 2.8, 95% CI: [2.2, 3.8]


In [ ]:
quantile_pairs = samples_df[['a[1]', 'a[2]', 'a[3]', 'a[4]', 'b[1]', 'b[2]', 'b[3]', 'b[4]', 's_y']].quantile([0.025, 0.975])
# print(quantile_pairs)
for param in ['a[1]', 'a[2]', 'a[3]', 'a[4]', 'b[1]', 'b[2]', 'b[3]', 'b[4]', 's_y']:
    mean_val = samples_df[param].mean()
    ci_low = quantile_pairs.loc[0.025, param]
    ci_high = quantile_pairs.loc[0.975, param]
    print(f"{param}: mean: {mean_val:.2f}, 95% CI: [{ci_low:.2f}, {ci_high:.2f}]")

a[1]: mean: 38.37, 95% CI: [35.15, 41.37]
a[2]: mean: 33.50, 95% CI: [30.12, 36.95]
a[3]: mean: 32.46, 95% CI: [25.45, 38.88]
a[4]: mean: 47.97, 95% CI: [30.10, 80.83]
b[1]: mean: 0.77, 95% CI: [0.59, 0.96]
b[2]: mean: 1.94, 95% CI: [1.68, 2.19]
b[3]: mean: 1.19, 95% CI: [0.88, 1.54]
b[4]: mean: 0.99, 95% CI: [-0.34, 1.71]
s_y: mean: 2.86, 95% CI: [2.21, 3.76]
